# Byte Pair Encoding (BPE)

**Descrição:**
Este notebook apresenta o funcionamento do **Byte Pair Encoding (BPE)**, um método de tokenização baseado em subpalavras amplamente utilizado em modelos de linguagem modernos. Diferentemente da tokenização por palavras, o BPE constrói tokens a partir de **padrões frequentes de caracteres**, permitindo representar palavras raras ou desconhecidas por meio da combinação de unidades menores.

**Objetivo:**
Demonstrar, de forma didática e passo a passo, como o algoritmo de Byte Pair Encoding:

* Aprende um vocabulário de subpalavras a partir de um corpus
* Reduz o problema de palavras fora do vocabulário (OOV)
* Cria uma representação compacta e eficiente do texto
* Converte texto em sequências de tokens subword (encode)
* Reconstrói o texto a partir desses tokens (decode)

O objetivo é entender por que o BPE é uma peça fundamental na preparação de texto para o treinamento de LLMs.

**Funcionamento:**

![Byte pair encoder](../../imagens/cap02/04_byte_pair_encode.png)

O funcionamento do BPE segue um processo iterativo de aprendizado e aplicação:

1. **Inicialização do vocabulário:**
   O texto do corpus é inicialmente representado como sequências de caracteres individuais, geralmente com um marcador de fim de palavra.

2. **Aprendizado por fusões (merges):**
   O algoritmo identifica o par de símbolos adjacentes mais frequente no corpus e os funde em um novo token.
   Esse processo é repetido iterativamente, expandindo gradualmente o vocabulário com subpalavras mais frequentes.

3. **Encode (tokenização):**
   Um novo texto é decomposto em subpalavras usando as regras de fusão aprendidas, garantindo que qualquer palavra possa ser representada, mesmo que nunca tenha aparecido no corpus original.

4. **Decode (reconstrução):**
   As sequências de subpalavras são combinadas para reconstruir o texto original, preservando a estrutura das palavras.

Esse mecanismo permite que modelos de linguagem equilibrem **tamanho de vocabulário**, **expressividade** e **generalização**, sendo a base de tokenizadores como os usados em GPT, BERT e outros LLMs.

**Tokenizadores reais:**

* **Open AI** https://platform.openai.com/tokenizer
* **Tiktokenizer** https://tiktokenizer.vercel.app/

## Implementação de um BPE

In [1]:
import re
from collections import Counter, defaultdict
from collections.abc import Iterable
from dataclasses import dataclass


@dataclass
class BPEResult:
    """Armazena resultados do treinamento do BPE (didático)."""

    vocab: dict[tuple[str, ...], int]
    merges: list[tuple[str, str]]
    token_to_id: dict[str, int]
    id_to_token: dict[int, str]


class BytePairEncoderDecoder:
    """
    Byte Pair Encoding (BPE) do zero, com passo a passo (didático).

    Ideia central (BPE clássico para subpalavras):
    - Representamos cada palavra como caracteres + marcador de fim de palavra (</w>)
    - Contamos pares adjacentes mais frequentes (bigramas de símbolos)
    - Iterativamente "fundimos" (merge) o par mais frequente em um novo símbolo
    - Guardamos a sequência de merges para tokenizar (encode) textos novos

    Observação:
    - Este é o BPE "clássico" baseado em palavras, suficiente para entender o mecanismo.
    - Em tokenizadores modernos (ex.: GPT-2), há detalhes adicionais (bytes, etc.),
      mas o coração do BPE (aprender merges frequentes) é o mesmo.
    BPE do zero (didático) **considerando pontuação**.

    Ajustes principais desta versão:
    - A tokenização inicial separa: palavras e pontuação ([,.!?;:])
    - Pontuação vira "tokens atômicos" (não recebem </w>)
    - Palavras recebem caracteres + </w> (fim de palavra), como no BPE clássico
    - O decode reconstrói o texto:
        * remove </w> e repõe espaços corretamente
        * NÃO coloca espaço antes de pontuação
    """

    def __init__(self, end_of_word: str = "</w>") -> None:
        if not isinstance(end_of_word, str) or not end_of_word:
            raise ValueError("`end_of_word` deve ser uma string não vazia.")
        self.end_of_word = end_of_word

        self.merges: list[tuple[str, str]] = []
        self.token_to_id: dict[str, int] = {}
        self.id_to_token: dict[int, str] = {}

        # Pontuação suportada (didático e suficiente pro seu exemplo)
        self._punct = {".", ",", "!", "?", ";", ":"}

    # ======================================================
    # 1) Tokenização: palavras + pontuação
    # ======================================================
    def _basic_tokenize_with_punct(self, text: str) -> list[str]:
        """
        Separa palavras e pontuação.

        Ex.: "O gato sobe no tapete." -> ["O", "gato", "sobe", "no", "tapete", "."]
        """
        if not isinstance(text, str):
            raise TypeError("O parâmetro `text` deve ser uma string.")
        if not text.strip():
            return []

        # Palavras (com acentos) OU pontuação isolada
        return re.findall(r"[A-Za-zÀ-ÿ0-9]+|[,.!?;:]", text)

    def _token_to_symbols(self, token: str) -> tuple[str, ...]:
        """
        Converte um token em símbolos para o BPE:
        - Se for pontuação: retorna (token,)
        - Se for palavra: retorna (c1, c2, ..., </w>)
        """
        if token in self._punct:
            return (token,)
        return tuple(list(token) + [self.end_of_word])

    # ======================================================
    # 2) Construção do vocab inicial (com pontuação)
    # ======================================================
    def build_initial_vocab(
        self, corpus: Iterable[str], verbose: bool = True
    ) -> dict[tuple[str, ...], int]:
        if corpus is None:
            raise TypeError("`corpus` não pode ser None.")

        corpus = list(corpus)
        if not corpus:
            raise ValueError("`corpus` precisa conter ao menos 1 frase/texto.")

        vocab: dict[tuple[str, ...], int] = defaultdict(int)

        if verbose:
            print("=== PASSO 1: Corpus bruto ===")
            for i, line in enumerate(corpus, start=1):
                print(f"{i}. {line}")
            print()

        if verbose:
            print("=== PASSO 2: Tokenização (palavras + pontuação) ===")

        all_tokens: list[str] = []
        for line in corpus:
            toks = self._basic_tokenize_with_punct(line)
            all_tokens.extend(toks)

        if verbose:
            print("Tokens extraídos:", all_tokens)
            print()

        if verbose:
            print(
                "=== PASSO 3: Vocabulário inicial (caracteres + </w> | pontuação atômica) ==="
            )

        for tok in all_tokens:
            sym = self._token_to_symbols(tok)
            vocab[sym] += 1

        if verbose:
            for sym, cnt in sorted(vocab.items(), key=lambda x: (-x[1], x[0])):
                print(f"{' '.join(sym)}  ->  {cnt}")
            print()

        return dict(vocab)

    # ======================================================
    # 3) Contagem de pares e merges (BPE clássico)
    # ======================================================
    @staticmethod
    def _get_pair_frequencies(
        vocab: dict[tuple[str, ...], int],
    ) -> Counter[tuple[str, str]]:
        pairs: Counter[tuple[str, str]] = Counter()
        for word_symbols, freq in vocab.items():
            if len(word_symbols) < 2:
                continue
            for i in range(len(word_symbols) - 1):
                pairs[(word_symbols[i], word_symbols[i + 1])] += freq
        return pairs

    @staticmethod
    def _merge_pair_in_word(
        word_symbols: tuple[str, ...], pair: tuple[str, str]
    ) -> tuple[str, ...]:
        a, b = pair
        merged_symbol = a + b

        new_symbols: list[str] = []
        i = 0
        while i < len(word_symbols):
            if (
                i < len(word_symbols) - 1
                and word_symbols[i] == a
                and word_symbols[i + 1] == b
            ):
                new_symbols.append(merged_symbol)
                i += 2
            else:
                new_symbols.append(word_symbols[i])
                i += 1

        return tuple(new_symbols)

    def train(
        self,
        corpus: Iterable[str],
        num_merges: int = 50,
        verbose: bool = True,
        top_pairs_to_show: int = 10,
    ) -> BPEResult:
        if not isinstance(num_merges, int) or num_merges <= 0:
            raise ValueError("`num_merges` deve ser um inteiro > 0.")

        vocab = self.build_initial_vocab(corpus, verbose=verbose)
        self.merges = []

        for step in range(1, num_merges + 1):
            pairs = self._get_pair_frequencies(vocab)
            if not pairs:
                if verbose:
                    print("Nenhum par restante para fundir. Encerrando.")
                break

            best_pair, best_freq = pairs.most_common(1)[0]

            if verbose:
                print(
                    f"=== PASSO 4.{step}: Contagem de pares (top {top_pairs_to_show}) ==="
                )
                for p, f in pairs.most_common(top_pairs_to_show):
                    print(f"{p} -> {f}")
                print()
                print(f"=== PASSO 5.{step}: Melhor par para merge ===")
                print(f"Par escolhido: {best_pair} (freq={best_freq})")
                print(f"Novo símbolo: '{best_pair[0] + best_pair[1]}'")
                print()

            new_vocab: dict[tuple[str, ...], int] = defaultdict(int)
            for word_symbols, freq in vocab.items():
                merged = self._merge_pair_in_word(word_symbols, best_pair)
                new_vocab[merged] += freq

            vocab = dict(new_vocab)
            self.merges.append(best_pair)

            if verbose:
                print(f"=== PASSO 6.{step}: Vocabulário após merge (amostra) ===")
                for sym, cnt in sorted(vocab.items(), key=lambda x: (-x[1], x[0]))[:15]:
                    print(f"{' '.join(sym)}  ->  {cnt}")
                if len(vocab) > 15:
                    print("... (mostrando apenas 15 entradas)")
                print()

        # Tokens finais = todos símbolos que aparecem no vocab final
        final_tokens: set[str] = set()
        for word_symbols in vocab.keys():
            final_tokens.update(word_symbols)

        final_tokens_sorted = sorted(final_tokens)
        self.token_to_id = {tok: i for i, tok in enumerate(final_tokens_sorted)}
        self.id_to_token = {i: tok for tok, i in self.token_to_id.items()}

        if verbose:
            print("=== PASSO 7: Tokens finais (vocabulário de subpalavras) ===")
            print(final_tokens_sorted)
            print()

        return BPEResult(
            vocab=vocab,
            merges=self.merges.copy(),
            token_to_id=self.token_to_id.copy(),
            id_to_token=self.id_to_token.copy(),
        )

    # ======================================================
    # 4) Encode / Decode (com pontuação)
    # ======================================================
    def _apply_merges_to_symbols(
        self, symbols: tuple[str, ...], verbose: bool = True
    ) -> tuple[str, ...]:
        if verbose:
            print("Símbolos iniciais:", symbols)

        for i, pair in enumerate(self.merges, start=1):
            new_symbols = self._merge_pair_in_word(symbols, pair)
            if new_symbols != symbols and verbose:
                print(f"Merge {i}: {pair} -> '{pair[0] + pair[1]}'")
                print("Antes:", symbols)
                print("Depois:", new_symbols)
                print()
            symbols = new_symbols

        if verbose:
            print("Símbolos finais (após merges):", symbols)
            print()

        return symbols

    def encode(self, text: str, verbose: bool = True) -> list[int]:
        if not self.token_to_id or not self.merges:
            raise ValueError("BPE não treinado. Execute `train()` antes de `encode()`.")

        if verbose:
            print("=== ENCODE (BPE com pontuação) ===")
            print("Texto:", text)
            print()

        tokens = self._basic_tokenize_with_punct(text)
        if verbose:
            print("Tokens (palavra/pontuação):", tokens)
            print()

        all_ids: list[int] = []
        for tok in tokens:
            if verbose:
                print(f"--- Token: '{tok}' ---")
            symbols = self._token_to_symbols(tok)
            final_symbols = self._apply_merges_to_symbols(symbols, verbose=verbose)

            ids = []
            for s in final_symbols:
                if s not in self.token_to_id:
                    raise ValueError(f"Token '{s}' não existe no vocabulário final.")
                ids.append(self.token_to_id[s])

            if verbose:
                print("Tokens BPE finais:", final_symbols)
                print("IDs:", ids)
                print()

            all_ids.extend(ids)

        if verbose:
            print("=== SEQUÊNCIA FINAL DE IDS ===")
            print(all_ids)
            print()

        return all_ids

    def decode(self, ids: list[int], verbose: bool = True) -> str:
        if not isinstance(ids, list) or any(not isinstance(i, int) for i in ids):
            raise TypeError("`ids` deve ser uma lista de inteiros (list[int]).")
        if not self.id_to_token:
            raise ValueError("BPE não treinado. Execute `train()` antes de `decode()`.")

        if verbose:
            print("=== DECODE (BPE com pontuação) ===")
            print("IDs:", ids)
            print()

        symbols = []
        for i in ids:
            if i not in self.id_to_token:
                raise ValueError(f"ID inválido: {i}")
            symbols.append(self.id_to_token[i])

        if verbose:
            print("Símbolos:", symbols)
            print()

        # Reconstrução:
        # - </w> vira "separador de palavra" (um espaço)
        # - pontuação não recebe espaço antes
        out_parts: list[str] = []
        current_word = ""

        def flush_word() -> None:
            nonlocal current_word
            if current_word:
                out_parts.append(current_word)
                current_word = ""

        for s in symbols:
            if s == self.end_of_word:
                flush_word()
            elif s in self._punct:
                # fecha palavra atual e anexa pontuação ao final do texto (sem espaço antes)
                flush_word()
                if out_parts:
                    out_parts[-1] = out_parts[-1] + s
                else:
                    # caso estranho: pontuação no começo
                    out_parts.append(s)
            else:
                # símbolo normal (subpalavra/char/merge)
                current_word += s

        flush_word()

        text = " ".join(out_parts).strip()

        if verbose:
            print("Texto reconstruído:", text)
            print()

        return text

## Geração do corpus

In [2]:
corpus = [
    "O gato sobe no tapete.",
    "O cachorro sobe na mesa.",
    "A aranha desce a parede, e o gato desce da mesa.",
]

bpe = BytePairEncoderDecoder(end_of_word="</w>")

# Treinar com poucas merges para ficar bem visível no passo a passo
resultado = bpe.train(corpus=corpus, num_merges=10, verbose=True, top_pairs_to_show=10)

=== PASSO 1: Corpus bruto ===
1. O gato sobe no tapete.
2. O cachorro sobe na mesa.
3. A aranha desce a parede, e o gato desce da mesa.

=== PASSO 2: Tokenização (palavras + pontuação) ===
Tokens extraídos: ['O', 'gato', 'sobe', 'no', 'tapete', '.', 'O', 'cachorro', 'sobe', 'na', 'mesa', '.', 'A', 'aranha', 'desce', 'a', 'parede', ',', 'e', 'o', 'gato', 'desce', 'da', 'mesa', '.']

=== PASSO 3: Vocabulário inicial (caracteres + </w> | pontuação atômica) ===
.  ->  3
O </w>  ->  2
d e s c e </w>  ->  2
g a t o </w>  ->  2
m e s a </w>  ->  2
s o b e </w>  ->  2
,  ->  1
A </w>  ->  1
a </w>  ->  1
a r a n h a </w>  ->  1
c a c h o r r o </w>  ->  1
d a </w>  ->  1
e </w>  ->  1
n a </w>  ->  1
n o </w>  ->  1
o </w>  ->  1
p a r e d e </w>  ->  1
t a p e t e </w>  ->  1

=== PASSO 4.1: Contagem de pares (top 10) ===
('e', '</w>') -> 7
('a', '</w>') -> 6
('o', '</w>') -> 5
('e', 's') -> 4
('d', 'e') -> 3
('O', '</w>') -> 2
('g', 'a') -> 2
('a', 't') -> 2
('t', 'o') -> 2
('s', 'o') -> 2



In [3]:
resultado.token_to_id

{',': 0,
 '.': 1,
 '</w>': 2,
 'A': 3,
 'O</w>': 4,
 'a': 5,
 'a</w>': 6,
 'c': 7,
 'd': 8,
 'e': 9,
 'e</w>': 10,
 'es': 11,
 'gato</w>': 12,
 'h': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'o</w>': 17,
 'p': 18,
 'r': 19,
 'sob': 20,
 't': 21}

## Exemplo de uso do encode (uma frase ainda não utilizada)

In [4]:
# Encode: veja como palavras viram subpalavras/bytes (aqui: símbolos BPE)
texto_novo = "A aranha sobe no gato."
ids = bpe.encode(texto_novo, verbose=True)

=== ENCODE (BPE com pontuação) ===
Texto: A aranha sobe no gato.

Tokens (palavra/pontuação): ['A', 'aranha', 'sobe', 'no', 'gato', '.']

--- Token: 'A' ---
Símbolos iniciais: ('A', '</w>')
Símbolos finais (após merges): ('A', '</w>')

Tokens BPE finais: ('A', '</w>')
IDs: [3, 2]

--- Token: 'aranha' ---
Símbolos iniciais: ('a', 'r', 'a', 'n', 'h', 'a', '</w>')
Merge 2: ('a', '</w>') -> 'a</w>'
Antes: ('a', 'r', 'a', 'n', 'h', 'a', '</w>')
Depois: ('a', 'r', 'a', 'n', 'h', 'a</w>')

Símbolos finais (após merges): ('a', 'r', 'a', 'n', 'h', 'a</w>')

Tokens BPE finais: ('a', 'r', 'a', 'n', 'h', 'a</w>')
IDs: [5, 19, 5, 15, 13, 6]

--- Token: 'sobe' ---
Símbolos iniciais: ('s', 'o', 'b', 'e', '</w>')
Merge 1: ('e', '</w>') -> 'e</w>'
Antes: ('s', 'o', 'b', 'e', '</w>')
Depois: ('s', 'o', 'b', 'e</w>')

Merge 9: ('s', 'o') -> 'so'
Antes: ('s', 'o', 'b', 'e</w>')
Depois: ('so', 'b', 'e</w>')

Merge 10: ('so', 'b') -> 'sob'
Antes: ('so', 'b', 'e</w>')
Depois: ('sob', 'e</w>')

Símbolos finai

## Exemplo de uso do decode (Uma frase ainda não utilizada)

In [5]:
# Decode: volta ao texto
texto_reconstruido = bpe.decode(ids, verbose=True)

=== DECODE (BPE com pontuação) ===
IDs: [3, 2, 5, 19, 5, 15, 13, 6, 20, 10, 15, 17, 12, 1]

Símbolos: ['A', '</w>', 'a', 'r', 'a', 'n', 'h', 'a</w>', 'sob', 'e</w>', 'n', 'o</w>', 'gato</w>', '.']

Texto reconstruído: A aranha</w>sobe</w>no</w>gato</w>.



## Este método é naturalmente robusto a palavras desconhecidas

In [6]:
# Encode: veja como palavras viram subpalavras/bytes (aqui: símbolos BPE)
texto_novo = "A aranha sobe no rato."
ids = bpe.encode(texto_novo, verbose=True)

=== ENCODE (BPE com pontuação) ===
Texto: A aranha sobe no rato.

Tokens (palavra/pontuação): ['A', 'aranha', 'sobe', 'no', 'rato', '.']

--- Token: 'A' ---
Símbolos iniciais: ('A', '</w>')
Símbolos finais (após merges): ('A', '</w>')

Tokens BPE finais: ('A', '</w>')
IDs: [3, 2]

--- Token: 'aranha' ---
Símbolos iniciais: ('a', 'r', 'a', 'n', 'h', 'a', '</w>')
Merge 2: ('a', '</w>') -> 'a</w>'
Antes: ('a', 'r', 'a', 'n', 'h', 'a', '</w>')
Depois: ('a', 'r', 'a', 'n', 'h', 'a</w>')

Símbolos finais (após merges): ('a', 'r', 'a', 'n', 'h', 'a</w>')

Tokens BPE finais: ('a', 'r', 'a', 'n', 'h', 'a</w>')
IDs: [5, 19, 5, 15, 13, 6]

--- Token: 'sobe' ---
Símbolos iniciais: ('s', 'o', 'b', 'e', '</w>')
Merge 1: ('e', '</w>') -> 'e</w>'
Antes: ('s', 'o', 'b', 'e', '</w>')
Depois: ('s', 'o', 'b', 'e</w>')

Merge 9: ('s', 'o') -> 'so'
Antes: ('s', 'o', 'b', 'e</w>')
Depois: ('so', 'b', 'e</w>')

Merge 10: ('so', 'b') -> 'sob'
Antes: ('so', 'b', 'e</w>')
Depois: ('sob', 'e</w>')

Símbolos finai

## Decode com a palavra desconhecida

In [7]:
# Decode: volta ao texto
texto_reconstruido = bpe.decode(ids, verbose=True)

=== DECODE (BPE com pontuação) ===
IDs: [3, 2, 5, 19, 5, 15, 13, 6, 20, 10, 15, 17, 19, 5, 21, 17, 1]

Símbolos: ['A', '</w>', 'a', 'r', 'a', 'n', 'h', 'a</w>', 'sob', 'e</w>', 'n', 'o</w>', 'r', 'a', 't', 'o</w>', '.']

Texto reconstruído: A aranha</w>sobe</w>no</w>rato</w>.



## O número de iterações influencia nos tokens gerados

### Iterações = 1

In [8]:
corpus = [
    "O gato sobe no tapete.",
    "O cachorro sobe na mesa.",
    "A aranha desce a parede.",
    "O gato desce da mesa.",
]

bpe = BytePairEncoderDecoder(end_of_word="</w>")
resultado = bpe.train(corpus=corpus, num_merges=1, verbose=False)
resultado.token_to_id

{'.': 0,
 '</w>': 1,
 'A': 2,
 'O': 3,
 'a': 4,
 'b': 5,
 'c': 6,
 'd': 7,
 'e': 8,
 'e</w>': 9,
 'g': 10,
 'h': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'r': 16,
 's': 17,
 't': 18}

In [9]:
texto_novo = "O gato sobe no tapete O cachorro sobe na mesa"
print(f"{len(texto_novo)=}")
ids = bpe.encode(texto_novo, verbose=False)
print(f"{ids=}, {len(ids)=}")

len(texto_novo)=45
ids=[3, 1, 10, 4, 18, 14, 1, 17, 14, 5, 9, 13, 14, 1, 18, 4, 15, 8, 18, 9, 3, 1, 6, 4, 6, 11, 14, 16, 16, 14, 1, 17, 14, 5, 9, 13, 4, 1, 12, 8, 17, 4, 1], len(ids)=43


In [10]:
taxa_compressao = 1 - (len(ids) / len(texto_novo))
taxa_compressao

0.0444444444444444

### Iterações = 20

In [11]:
bpe = BytePairEncoderDecoder(end_of_word="</w>")
resultado = bpe.train(corpus=corpus, num_merges=20, verbose=False)
resultado.token_to_id

{'.': 0,
 '</w>': 1,
 'A': 2,
 'O</w>': 3,
 'a': 4,
 'a</w>': 5,
 'ar': 6,
 'c': 7,
 'd': 8,
 'desce</w>': 9,
 'e': 10,
 'e</w>': 11,
 'gato</w>': 12,
 'h': 13,
 'mesa</w>': 14,
 'n': 15,
 'no</w>': 16,
 'o': 17,
 'o</w>': 18,
 'p': 19,
 'r': 20,
 'sobe</w>': 21,
 't': 22,
 'tap': 23}

In [12]:
texto_novo = "O gato sobe no tapete O cachorro sobe na mesa."
print(f"{len(texto_novo)=}")
ids = bpe.encode(texto_novo, verbose=False)
print(f"{ids=}, {len(ids)=}")

len(texto_novo)=46
ids=[3, 12, 21, 16, 23, 10, 22, 11, 3, 7, 4, 7, 13, 17, 20, 20, 18, 21, 15, 5, 14, 0], len(ids)=22


In [13]:
taxa_compressao = 1 - (len(ids) / len(texto_novo))
taxa_compressao

0.5217391304347826

### Iterações = 40

In [14]:
bpe = BytePairEncoderDecoder(end_of_word="</w>")
resultado = bpe.train(corpus=corpus, num_merges=40, verbose=False)
resultado.token_to_id

{'.': 0,
 'A</w>': 1,
 'O</w>': 2,
 'a</w>': 3,
 'aranha</w>': 4,
 'cachorro</w>': 5,
 'd': 6,
 'desce</w>': 7,
 'gato</w>': 8,
 'mesa</w>': 9,
 'na</w>': 10,
 'no</w>': 11,
 'parede</w>': 12,
 'sobe</w>': 13,
 'tapete</w>': 14}

In [15]:
texto_novo = "O gato sobe no tapete O cachorro sobe na mesa"
print(f"{len(texto_novo)=}")
ids = bpe.encode(texto_novo, verbose=False)
print(f"{ids=}, {len(ids)=}")

len(texto_novo)=45
ids=[2, 8, 13, 11, 14, 2, 5, 13, 10, 9], len(ids)=10


In [16]:
taxa_compressao = 1 - (len(ids) / len(texto_novo))
taxa_compressao

0.7777777777777778

## Taxa de compressão para textos muito repetitivos

In [17]:
corpus = [
    # Presente do Indicativo
    "ando",
    "andas",
    "anda",
    "andamos",
    "andais",
    "andam",
    # Pretérito Perfeito do Indicativo
    "andei",
    "andaste",
    "andou",
    "andamos",
    "andastes",
    "andaram",
    # Pretérito Imperfeito do Indicativo
    "andava",
    "andavas",
    "andava",
    "andávamos",
    "andáveis",
    "andavam",
    # Futuro do Presente do Indicativo
    "andarei",
    "andarás",
    "andará",
    "andaremos",
    "andareis",
    "andarão",
    # Futuro do Pretérito (Condicional)
    "andaria",
    "andarias",
    "andaria",
    "andaríamos",
    "andaríeis",
    "andariam",
    # Presente do Subjuntivo
    "ande",
    "andes",
    "ande",
    "andemos",
    "andeis",
    "andem",
    # Imperativo Afirmativo
    "anda",
    "ande",
    "andemos",
    "andai",
    "andem",
]


bpe = BytePairEncoderDecoder(end_of_word="</w>")
resultado = bpe.train(corpus=corpus, num_merges=5, verbose=True)
resultado.token_to_id

=== PASSO 1: Corpus bruto ===
1. ando
2. andas
3. anda
4. andamos
5. andais
6. andam
7. andei
8. andaste
9. andou
10. andamos
11. andastes
12. andaram
13. andava
14. andavas
15. andava
16. andávamos
17. andáveis
18. andavam
19. andarei
20. andarás
21. andará
22. andaremos
23. andareis
24. andarão
25. andaria
26. andarias
27. andaria
28. andaríamos
29. andaríeis
30. andariam
31. ande
32. andes
33. ande
34. andemos
35. andeis
36. andem
37. anda
38. ande
39. andemos
40. andai
41. andem

=== PASSO 2: Tokenização (palavras + pontuação) ===
Tokens extraídos: ['ando', 'andas', 'anda', 'andamos', 'andais', 'andam', 'andei', 'andaste', 'andou', 'andamos', 'andastes', 'andaram', 'andava', 'andavas', 'andava', 'andávamos', 'andáveis', 'andavam', 'andarei', 'andarás', 'andará', 'andaremos', 'andareis', 'andarão', 'andaria', 'andarias', 'andaria', 'andaríamos', 'andaríeis', 'andariam', 'ande', 'andes', 'ande', 'andemos', 'andeis', 'andem', 'anda', 'ande', 'andemos', 'andai', 'andem']

=== PASSO 3: 

{'</w>': 0,
 'a': 1,
 'and': 2,
 'anda': 3,
 'andar': 4,
 'e': 5,
 'i': 6,
 'm': 7,
 'o': 8,
 's': 9,
 's</w>': 10,
 't': 11,
 'u': 12,
 'v': 13,
 'á': 14,
 'ã': 15,
 'í': 16}

In [18]:
texto_novo = "ando andas"
print(f"{len(texto_novo)=}")
ids = bpe.encode(texto_novo, verbose=False)
print(f"{ids=}, {len(ids)=}")

len(texto_novo)=10
ids=[2, 8, 0, 3, 10], len(ids)=5


In [19]:
taxa_compressao = 1 - (len(ids) / len(texto_novo))
taxa_compressao

0.5